In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@2fb564c4

In [2]:
import spark.implicits._
case class Student(student_id: Int, student_name: String)
case class Subject(subject_name: String)
case class Examination(student_id: Int, subject_name: String)

val studentsData = Seq(
  (1, "Alice"), (2, "Bob"), (13, "John"), (6, "Alex")
)

val subjectsData = Seq(
  ("Math"), ("Physics"), ("Programming")
)

val examinationsData = Seq(
  (1, "Math"), (1, "Physics"), (1, "Programming"), (2, "Programming"),
  (1, "Physics"), (1, "Math"), (13, "Math"), (13, "Programming"),
  (13, "Physics"), (2, "Math"), (1, "Math")
)

val examinations = examinationsData.toDF("student_id", "subject_name").as[Examination]
val students = studentsData.toDF("student_id", "student_name").as[Student]
val subjects = subjectsData.toDF("subject_name").as[Subject]

examinations.show()
students.show()
subjects.show()


+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+

+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|        13|        John|
|         6|        Alex|
+----------+------------+

+------------+
|subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+



import spark.implicits._

defined class Student
defined class Subject
defined class Examination
studentsData: Seq[(Int, String)] = List(
  (1, "Alice"),
  (2, "Bob"),
  (13, "John"),
  (6, "Alex")
)
subjectsData: Seq[String] = List("Math", "Physics", "Programming")
examinationsData: Seq[(Int, String)] = List(
  (1, "Math"),
  (1, "Physics"),
  (1, "Programming"),
  (2, "Programming"),
  (1, "Physics"),
  (1, "Math"),
  (13, "Math"),
  (13, "Programming"),
  (13, "Physics"),
  (2, "Math"),
  (1, "Math")
)
examinations: Dataset[Examination] = [student_id: int, subject_name: string]
students: Dataset[Student] = [student_id: int, student_name: string]
subjects: Dataset[Subject] = [subject_name: string]

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

| student_id | student_name | subject_name | attended_exams |
|------------|--------------|--------------|----------------|
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |

In [3]:
val countDF = examinations.groupBy("student_id", "subject_name")
    .agg(count("subject_name").as("attended_exams"))
val studentsSubjects = students.join(subjects)
    .select(students("student_id"), students("student_name"), subjects("subject_name"))
studentsSubjects.join(
        countDF, 
        (studentsSubjects("student_id") === countDF("student_id")) &&
        (studentsSubjects("subject_name") === countDF("subject_name")), 
        "left_outer"
    )
    .select(studentsSubjects("student_id"), studentsSubjects("student_name"), studentsSubjects("subject_name"), coalesce(countDF("attended_exams"), lit(0)).as("attended_exams"))
    .sort(studentsSubjects("student_id"), studentsSubjects("student_name"))
    .show()

+----------+------------+------------+--------------+
|student_id|student_name|subject_name|attended_exams|
+----------+------------+------------+--------------+
|         1|       Alice|     Physics|             2|
|         1|       Alice|        Math|             3|
|         1|       Alice| Programming|             1|
|         2|         Bob|     Physics|             0|
|         2|         Bob|        Math|             1|
|         2|         Bob| Programming|             1|
|         6|        Alex|        Math|             0|
|         6|        Alex| Programming|             0|
|         6|        Alex|     Physics|             0|
|        13|        John|        Math|             1|
|        13|        John|     Physics|             1|
|        13|        John| Programming|             1|
+----------+------------+------------+--------------+



countDF: DataFrame = [student_id: int, subject_name: string ... 1 more field]
studentsSubjects: DataFrame = [student_id: int, student_name: string ... 1 more field]

In [4]:
spark.stop()